# Fabric Notebook: Access Dataverse (Power Apps Data Platform)
This notebook demonstrates how to authenticate with Dataverse using a Service Principal (Client Credentials Flow) and query the `accounts` table using the Web API. This replaces the functionality of the local PowerShell scripts `get-Dataverse-token.ps1` and `get-Dataverse-accounts.ps1`.

### Prerequisites
- A Service Principal (App Registration) with access to the Dataverse environment.
- The `msal` library (Microsoft Authentication Library) installed in the environment.

In [ ]:
# 1. Install Required Libraries
# msal (Microsoft Authentication Library) is recommended for Python authentication
%pip install msal requests

In [ ]:
# 2. Configuration (Replace with your own values or use mssparkutils.credentials.getSecret)

# Environment URL (e.g., https://org1234.crm.dynamics.com)
dataverse_env_url = "https://orgd2bf3532.crm4.dynamics.com"

# Credentials (Use Key Vault in production!)


# Construct the Token Scope
# Dataverse requires the scope to be the Environment URL + "/.default"
if not dataverse_env_url.endswith("/"):
    token_scope = f"{dataverse_env_url}/.default"
else:
    token_scope = f"{dataverse_env_url}.default"

authority_url = f"https://login.microsoftonline.com/{tenant_id}"

print(f"Target Environment: {dataverse_env_url}")
print(f"Auth Scope: {token_scope}")

In [ ]:
# 3. Authenticate and Get Token (Using MSAL)
import msal
import requests
import json

app = msal.ConfidentialClientApplication(
    client_id, 
    authority=authority_url,
    client_credential=client_secret
)

# Acquire token
result = app.acquire_token_for_client(scopes=[token_scope])

if "access_token" in result:
    token = result['access_token']
    print("Authentication successful. Token acquired.")
else:
    print("Authentication failed.")
    print(result.get("error"))
    print(result.get("error_description"))
    raise Exception("Could not retrieve access token")

In [ ]:
# 4. Query Dataverse Web API (Accounts)

# Construct URI
api_version = "v9.2"
entity_name = "accounts"
# Using OData query options to limit columns (select) and row count (top)
query_options = "?$select=name,accountnumber&$top=5" 

# Ensure no trailing slash for base URL
base_url = dataverse_env_url.rstrip("/")
request_uri = f"{base_url}/api/data/{api_version}/{entity_name}{query_options}"

# Set Headers
headers = {
    "Authorization": f"Bearer {token}",
    "OData-MaxVersion": "4.0",
    "OData-Version": "4.0",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Prefer": "odata.include-annotations=*" 
}

print(f"Querying URI: {request_uri}")

In [ ]:
# 5. Execute Request and Process Response

try:
    response = requests.get(request_uri, headers=headers)
    
    # Check for HTTP errors
    response.raise_for_status()

    # Parse JSON
    data = response.json()

    if "value" in data and len(data["value"]) > 0:
        accounts = data["value"]
        print(f"Successfully retrieved {len(accounts)} accounts.")
        
        # Display the First Account Details
        first_account = accounts[0]
        print("\n--- First Account Details ---")
        print(f"Account Name: {first_account.get('name')}")
        print(f"Account Number: {first_account.get('accountnumber')}")
        
        # Display All in a nicer format (Pandas DataFrame)
        import pandas as pd
        df = pd.DataFrame(accounts)
        display(df[['name', 'accountnumber', 'accountid']]) # Requires Fabric Notebook display

    else:
        print("No accounts found.")
        print("Full Response:")
        print(json.dumps(data, indent=2))

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")
    print(response.text)
except Exception as e:
    print(f"Error: {e}")